In [ ]:
### userLogin
import json
import boto3


def login(userid,userpassword):

    dynamodb=boto3.resource(service_name='dynamodb',
                            aws_access_key_id="",
                            aws_secret_access_key='',
                            region_name="us-east-1",
                         )

    loginTable=dynamodb.Table('userLogin')
#     print(table.creation_date_time)

    res=loginTable.get_item(Key={'userID':str(userid)})
    print(res)
    
    if 'Item' in res:
        userInfo=res['Item']
        if userInfo['password']==str(userpassword):
            return 1 #login
        else:
            return -1 #wrong password
            # print("wrong password")
    else:
        newitem = {
            "userID": str(userid),
            "password": str(userpassword)
        } 
        loginTable.put_item(Item=newitem)
        
        user2caleTable = dynamodb.Table('user2cale')
        calendar = {}
        for i in range(1,8):
            calendar[i] = {}
            for time in range(9, 23):
                calendar[i][time] = 'None'
        
        user2caleTable.put_item(
            Item={
                "userID": str(userid),
                "calendar": json.dumps(calendar),
                "meetings": ''
            }
        )
        
        
        
        return 0 #register
        # print("register done")



def lambda_handler(event, context):
    # TODO implement
    userid = event["queryStringParameters"]['uID']
    print(userid)
    userpassword=event["queryStringParameters"]['uPSWD']
    print(userpassword)
    login_status=login(userid,userpassword)
    
    if login_status==1:
        return{
                'statusCode': 200,
                'headers': {
                    'Access-Control-Allow-Headers': 'Content-Type',
                    'Access-Control-Allow-Origin': '*',
                    'Access-Control-Allow-Methods': 'OPTIONS,POST,GET',
                },
                'body': json.dumps('login')
        }
        

    elif login_status==0:
        return{
                'statusCode': 200,
                'headers': {
                    'Access-Control-Allow-Headers': 'Content-Type',
                    'Access-Control-Allow-Origin': '*',
                    'Access-Control-Allow-Methods': 'OPTIONS,POST,GET',
                },
                'body': json.dumps('registered')
        }
    
    elif login_status==-1:
        return{
                'statusCode': 200,
                'headers': {
                    'Access-Control-Allow-Headers': 'Content-Type',
                    'Access-Control-Allow-Origin': '*',
                    'Access-Control-Allow-Methods': 'OPTIONS,POST,GET',
                },
                'body': json.dumps('wrong')
        }

In [ ]:
#initializeMeeting
import json
import boto3
import math

db=boto3.resource(service_name='dynamodb',
                    aws_access_key_id="",
                    aws_secret_access_key='',
                    region_name="us-east-1",
                )


user2caletable = db.Table("user2cale")
meetingtable = db.Table("meetingSchedule")


def recommend(host,users, duration):
    #users string
    allusers=users+','+host
    participant=allusers.split(',')
    # print("participant:",participant)
    # print("type participant:",type(participant))
    user_calendars = []
    for user in participant:
        print(user)
        x=user2caletable.get_item(Key = {'userID':str(user)})
        print(x)
        temp = json.loads(user2caletable.get_item(Key = {'userID':str(user)})['Item']['calendar'])
        user_calendars.append(temp)
    toreturn = {}
    for i in range(1,8):
        toreturn[i] = {}
        for time in range(9, 23):
            toreturn[i][time] = 0

    for i in user_calendars:
        for j in range(1,8):
            for time in range(9, 23 - math.ceil(duration)+1):
                avaiable = True
                for hour in range(math.ceil(duration)):
                    if i[str(j)][str(time+hour)] != 'None':
                        avaiable = False
                if avaiable:
                    toreturn[j][time] += 1

    recommendation = []
    for i in range(1,8):
        for time in range(9,23):
            if toreturn[i][time] == len(participant):
                recommendation.append((i,time))
    return recommendation
    

    
def create_meeting(users,recommended_time,current_meeting_time, duration,host,location,description):
    # users string
    #participant list
    allusers=users+','+host
    participant = allusers.split(',')
    print("participant:",participant)
    print("type participant:",type(participant))
    meeting_name = str(int(meetingtable.scan()['Count']) + 1)
    for user in participant:
        temp = json.loads(user2caletable.get_item(Key = {'userID':str(user)})['Item']['calendar'])
        meetings = user2caletable.get_item(Key = {'userID':str(user)})['Item']['meetings']
        
        temp[str(current_meeting_time[0])][str(current_meeting_time[1])] = meeting_name
        
        for i in range(duration):
            temp[str(current_meeting_time[0])][str(current_meeting_time[1]+i)] = meeting_name

        user2caletable.delete_item(Key = {'userID':str(user)})

        if len(meetings) == 0:
            meeting_str = str(meeting_name)
        else:
            meeting_str = meetings + ','+ str(meeting_name)
        newitem = {
            "userID": str(user),
            "calendar": json.dumps(temp),
            'meetings': meeting_str
        }
        user2caletable.put_item(Item = newitem)
    potential_time = {}

    for data in recommended_time:
        temp = potential_time.get(data[0],{})
        temp[data[1]] = 0
        potential_time[data[0]] = temp

    meetingtable.put_item(Item={
        'meetID': meeting_name,
        'host':host,
        'potential_time':json.dumps(potential_time),
        'current_meeting_time':current_meeting_time,
        'highest_votes': 0,
        'voted_user': '',
        'user':allusers,
        'duration': duration,
        'location':location,
        'description':description
    })
    for i in range(duration+1):
        delete_time_from_meetings(meeting_name,current_meeting_time[0],current_meeting_time[1]+i)
        
    
    
def delete_time_from_meetings(meetID,day,time):
    count = int(meetingtable.scan()['Count'])+1
    
    for i in range(1,count):
        if str(i) != meetID:
            cur_meeting = meetingtable.get_item(Key = {'meetID':str(i)})
            if 'Item' in cur_meeting:
                cur_meeting_info=cur_meeting['Item']
                potential_time = json.loads(cur_meeting_info['potential_time'])
                if str(day) in potential_time:
                    if str(time) in potential_time[str(day)]:
                        del potential_time[str(day)][str(time)]
                cur_meeting_info['potential_time'] = json.dumps(potential_time)
                
                meetingtable.delete_item(Key = {'meetID':str(i)})
                meetingtable.put_item(Item=cur_meeting_info)

            
            
def lambda_handler(event, context):
    inputdata=json.loads(event['body'])
    
    users=inputdata['users'] #str
    duration=inputdata['duration'] #int
    host=inputdata['host'] #str
    location=inputdata['location'] #str
    description=inputdata['description'] #str
    
    recommended_time = recommend(host,users,duration)   #recommend test successfully
    print("recommended_time:",recommended_time)
    current_meeting_time = recommended_time[0]
    print(current_meeting_time)
    
    
    create_meeting(users,recommended_time,current_meeting_time, duration,host,location,description)
    
 
    # TODO implement
    return {
        'statusCode': 200,
        'headers': {
                    'Access-Control-Allow-Headers': 'Content-Type',
                    'Access-Control-Allow-Origin': '*',
                    'Access-Control-Allow-Methods': 'OPTIONS,POST,GET',
                },
        'body': json.dumps('Create Successfully!')
    }


In [ ]:
#voteMeetings
import json
import boto3

db=boto3.resource(service_name='dynamodb',
                    aws_access_key_id="",
                    aws_secret_access_key='',
                    region_name="us-east-1",
                )


user2caletable = db.Table("user2cale")
meetingtable = db.Table("meetingSchedule")

def votes(user, meetID, day, time):
    n = len(day)
    temp_meeting = meetingtable.get_item(Key = {'meetID':meetID})['Item']
    
    for i in range(n):
        potential_time = json.loads(temp_meeting['potential_time']) #decimal list
        potential_time[str(day[i])][str(time[i])] = potential_time[str(day[i])][str(time[i])] + 1
        temp_meeting['highest_votes'] = max(temp_meeting['highest_votes'], potential_time[str(day[i])][str(time[i])])

    recommendation = []
    for i in range(1,8):
        if str(i) in potential_time:
            for time in range(9,23):
                if str(time) in potential_time[str(i)]:
                    if potential_time[str(i)][str(time)] == temp_meeting['highest_votes']:
                        recommendation.append((i,time))
    previous_meeting_time = temp_meeting['current_meeting_time']
    temp_meeting['current_meeting_time'] = str(recommendation[0][0]) + ',' + str(recommendation[0][1])
    
    duration = int(temp_meeting['duration'])
    for i in range(duration+1):
        delete_time_from_meetings(meetID,recommendation[0][0],recommendation[0][1]+i)
    for i in range(duration):
        if type(previous_meeting_time)==list:
            add_time_from_meetings(meetID,int(previous_meeting_time[0]),int(previous_meeting_time[1])+i)
        else:
            previous_meeting_time_list=previous_meeting_time.split(',')
            print(previous_meeting_time_list)
            add_time_from_meetings(meetID,int(previous_meeting_time_list[0]),int(previous_meeting_time_list[1])+i)
    
    if len(temp_meeting['voted_user']) != 0:
        temp_meeting['voted_user'] =  temp_meeting['voted_user'] + ',' + user
    else:
        temp_meeting['voted_user'] = user
    meetingtable.delete_item(Key = {'meetID':str(meetID)})
    meetingtable.put_item(Item=temp_meeting)
    for i in temp_meeting['user'].split(','):
        update_user(i,meetID)
    
    
                           
def update_user(userID,meetID):
    temp_user = user2caletable.get_item(Key={'userID':userID})['Item']
    calendar = json.loads(temp_user['calendar'])
    for i in range(1,8):
        for time in range(9, 23):
            if calendar[str(i)][str(time)] == meetID:
                calendar[str(i)][str(time)] = 'None'
    
    temp_meeting = meetingtable.get_item(Key = {'meetID':meetID})['Item']

    day,time = temp_meeting['current_meeting_time'].split(',')
    day,time = int(day),int(time)
    duration = int(temp_meeting['duration'])
    
    for i in range(duration):
        calendar[str(day)][str(time+i)] = meetID
    temp_user['calendar'] = json.dumps(calendar)
    
    user2caletable.put_item(Item=temp_user)
                           
                           
def add_time_from_meetings(meetID, day, time):
    count = int(meetingtable.scan()['Count'])+1
    
    for i in range(1,count):
        if str(i) != meetID:
            cur_meeting = meetingtable.get_item(Key = {'meetID':str(i)})
            if 'Item' in cur_meeting:
                cur_meeting_info=cur_meeting['Item']
                potential_time = json.loads(cur_meeting_info['potential_time'])
                if str(day) in potential_time:
                    if str(time) not in potential_time[str(day)]:
                        potential_time[str(day)][str(time)] = 0
                    
                cur_meeting_info['potential_time'] = json.dumps(potential_time)
                meetingtable.delete_item(Key = {'meetID':str(i)})
                meetingtable.put_item(Item=cur_meeting_info)
            
            
def delete_time_from_meetings(meetID,day,time):
    count = int(meetingtable.scan()['Count'])+1
    
    for i in range(1,count):
        if str(i) != meetID:
            cur_meeting = meetingtable.get_item(Key = {'meetID':str(i)})
            if 'Item' in cur_meeting:
                cur_meeting_info=cur_meeting['Item']
                potential_time = json.loads(cur_meeting_info['potential_time'])
                if str(day) in potential_time:
                    if str(time) in potential_time[str(day)]:
                        del potential_time[str(day)][str(time)]
                cur_meeting_info['potential_time'] = json.dumps(potential_time)
            
                meetingtable.delete_item(Key = {'meetID':str(i)})
                meetingtable.put_item(Item=cur_meeting_info)
            

def lambda_handler(event, context):
    #voting input sample
    # user='Moni'#string
    # meetID='9' #string
    # day=[3,4] #int list
    # time=[18,9] #int list
    
    inputdata=json.loads(event['body'])
    
    user=inputdata['user'] #str
    meetID=inputdata['meetID'] #int
    daystr=inputdata['day'] #str
    timestr=inputdata['time'] #str
    
    daylist=daystr.split(',')
    timelist=timestr.split(',')
    day=list(map(int, daylist))
    time=list(map(int,timelist))
    
    # print(user)
    # print(meetID)
    # print(day)
    # print(time)
    
    votes(user, meetID, day, time)
    
    
    # TODO implement
    return {
        'statusCode': 200,
        'headers': {
                    'Access-Control-Allow-Headers': 'Content-Type',
                    'Access-Control-Allow-Origin': '*',
                    'Access-Control-Allow-Methods': 'OPTIONS,POST,GET',
                },
        'body': json.dumps('voted successfully')
    }


In [ ]:
#setUsertime
import json
import boto3

db=boto3.resource(service_name='dynamodb',
                    aws_access_key_id="",
                    aws_secret_access_key='',
                    region_name="us-east-1",
                )


user2caletable = db.Table("user2cale")
# meetingtable = db.Table("meetingSchedule")


def update_user_calendar(user, day, time):
    day = day.split(',')
    time = time.split(',')
    user = user2caletable.get_item(Key={'userID':user})['Item']
    print(user)
    n = len(day)
    
    calendar = {}
    for i in range(1,8):
        calendar[str(i)] = {}
        for j in range(9, 23):
            calendar[str(i)][str(j)] = 'Filled'
    print(calendar)
    for i in range(n):
        calendar[str(day[i])][str(time[i])] = 'None'
    user['calendar'] = json.dumps(calendar)
    user2caletable.put_item(Item=user)

def lambda_handler(event, context):
    # user='Amaya'
    # day='1,1,1,1,1'
    # time='9,10,11,12,13'
    
    inputdata=json.loads(event['body'])
    
    user=inputdata['user'] #str
    daystr=inputdata['day'] #str
    timestr=inputdata['time'] #str
    
    update_user_calendar(user, daystr, timestr)
    # TODO implement
    return {
        'statusCode': 200,
        'headers': {
                    'Access-Control-Allow-Headers': 'Content-Type',
                    'Access-Control-Allow-Origin': '*',
                    'Access-Control-Allow-Methods': 'OPTIONS,POST,GET',
                },
        'body': json.dumps('Choose time successfully')
    }


In [ ]:
#getRecommend
import json
import boto3

db=boto3.resource(service_name='dynamodb',
                    aws_access_key_id="",
                    aws_secret_access_key='',
                    region_name="us-east-1",
                )

# user2caletable = db.Table("user2cale")
meetingtable = db.Table("meetingSchedule")

def getRecommend(meetid):
    recommend={}
    cur_meeting=meetingtable.get_item(Key = {'meetID':str(meetid)})
    if 'Item' in cur_meeting:
        cur_meeting_info=cur_meeting['Item']
        available_time_str=cur_meeting_info['potential_time']
        print(available_time_str)
        available_time_json=json.loads(available_time_str)
        print(available_time_json)
        for i in available_time_json.keys():
            tmp=[]
            for j in available_time_json[i]:
                tmp.append(str(j))
                tmp_s=",".join(tmp)
            recommend[str(i)]=tmp_s
    return str(recommend)

def lambda_handler(event, context):
    meetid= event["queryStringParameters"]['mID']
    # meetid='3'
    print(meetid)
    recommend=getRecommend(meetid)
    print(recommend)
 
    
    # TODO implement
    return {
        'statusCode': 200,
        'headers': {
                    'Access-Control-Allow-Headers': 'Content-Type',
                    'Access-Control-Allow-Origin': '*',
                    'Access-Control-Allow-Methods': 'OPTIONS,POST,GET',
                },
        'body': json.dumps(recommend)
    }


In [ ]:
#findAllusers
import json
import boto3

db=boto3.resource(service_name='dynamodb',
                    aws_access_key_id="",
                    aws_secret_access_key='',
                    region_name="us-east-1",
                )
                
usertable= db.Table("userLogin")

def findAllUsers():
    user=[]
    for i in usertable.scan()['Items']:
        user.append(i['userID'])
    return user


def lambda_handler(event, context):
    user=findAllUsers()
    # TODO implement
    return {
        'statusCode': 200,
        'headers': {
                    'Access-Control-Allow-Headers': 'Content-Type',
                    'Access-Control-Allow-Origin': '*',
                    'Access-Control-Allow-Methods': 'OPTIONS,POST,GET',
                },
        'body': json.dumps(user)
    }


In [ ]:
#sendEmail
import json
import boto3
import os
import email
import re


session = boto3.Session()
s3 = session.client('s3')
ses = session.client('ses')


def send_reply(sender,receiver,sender_name,email_subject,email_msg): 
    reply=email_msg+'\n'+sender_name
    content = ses.send_email(
        Destination={'ToAddresses': receiver},
        Message={
            'Body': {
                'Text': {
                    'Charset': 'UTF-8',
                    'Data': reply,
                },
            },
            'Subject': {
                'Charset': 'UTF-8',
                'Data': email_subject,
            },
        },
        Source=str(sender),
    )
    print("sent")
    
    

def lambda_handler(event, context):
    # TODO implement
    print("event: ", event)
    
    receiver=["ws2591@columbia.edu","jc5657@columbia.edu","yl4850@columbia.edu","bw2712@columbia.edu"]
    sender="wemeet@jiaqing.xyz"
    
    inputdata=json.loads(event['body'])
    sender_name=inputdata['name']
    email_subject=inputdata['subject']
    email_msg=inputdata['message']
    
    send_reply(sender,receiver,sender_name,email_subject,email_msg)
    
    
    return {
        'statusCode': 200,
        'headers': {
                    'Access-Control-Allow-Headers': 'Content-Type',
                    'Access-Control-Allow-Origin': '*',
                    'Access-Control-Allow-Methods': 'OPTIONS,POST,GET',
                },
        'body': json.dumps('Succeed')
    }



In [ ]:
#showUpcomingMeetings
import json
import boto3

dynamodb=boto3.resource(service_name='dynamodb',
                        aws_access_key_id="",
                        aws_secret_access_key='',
                        region_name="us-east-1",
                    )

def findUsersMeetings(user):
    #user:string
    user2caletable=dynamodb.Table('user2cale')
    res=user2caletable.get_item(Key = {'userID':str(user)})
    if 'Item' in res:
        userCale=res['Item']
        meetingstr=userCale['meetings']
        return meetingstr
    else:
        return None

def findMeetingInfo(user,meetings):
    meetingtable=dynamodb.Table('meetingSchedule')
    meetings_list=meetings.split(',') #list of string
    i=1
    meeting_dict={}
    for meeting in meetings_list:
        tmp_dict={}
        cur_name='Meeting{}'.format(i)
        cur_meeting = meetingtable.get_item(Key = {'meetID':str(meeting)})
        if 'Item' in cur_meeting:
            tmp_dict['meetid']=cur_meeting['Item']['meetID']
            tmp_dict['duration']=str(cur_meeting['Item']['duration'])
            tmp_dict['host']=cur_meeting['Item']['host']
            tmp_dict['participants']=cur_meeting['Item']['user']
            cur_meeting_time=cur_meeting['Item']['current_meeting_time']
            if cur_meeting_time !='None':
                print(cur_meeting_time)
                if type(cur_meeting_time)==list:
                    daytime=cur_meeting_time[0]
                    hourtime=cur_meeting_time[1]
                else:
                    time_list=cur_meeting_time.split(',')
                    daytime=time_list[0]
                    hourtime=time_list[1]
                tmp_time=(int(daytime),int(hourtime))
                print(tmp_time)
                tmp_dict['current_set_time']=tmp_time
            tmp_dict['location']=cur_meeting['Item']['location']
            tmp_dict['description']=cur_meeting['Item']['description']
            voted=cur_meeting['Item']['voted_user']
            print("voted:",voted)
            voted_list=voted.split(',')
            print("voted_list:",voted_list)
            if user in voted_list:
                tmp_dict['vote_status']='Y'
            else:
                tmp_dict['vote_status']='N'
        meeting_dict[cur_name]=tmp_dict
        i+=1
    print("meeting_dict:",meeting_dict)
    print("meeting_dict type:",type(meeting_dict))
    return meeting_dict

            
def lambda_handler(event, context):
    user = event["queryStringParameters"]['uID']
    meetings=findUsersMeetings(user)
    print(meetings)
    
    
    meeting_dict=findMeetingInfo(user,meetings)
    print(meeting_dict)
    
    # TODO implement
    return {
        'statusCode': 200,
        'headers': {
                    'Access-Control-Allow-Headers': 'Content-Type',
                    'Access-Control-Allow-Origin': '*',
                    'Access-Control-Allow-Methods': 'OPTIONS,POST,GET',
                },
        'body': json.dumps(meeting_dict)
    }
